In my first attempt at Logistic Regression I used all the numeric features, but now I want to exclude information from the contest that has already started.  

In [1]:
# import the data
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

writers = pd.read_csv("../clean data/user_summary_no2015.csv", index_col=0)
writers.head()

,Writer Name,Member Length,LifetimeWordCount,url,Age,Birthday,Favorite books or authors,Favorite noveling music,Hobbies,Location,...,Expected Max Submission,Expected Max Day,Expected Std Submissions,Expected Consec Subs,FW Total,FW Sub,FH Total,FH Sub,SH Total,SH Sub
0,Nicaless,2,50919,http://nanowrimo.org/participants/nicaless,24,December 20,"Ursula Le Guin, J.K.","Classical, Musicals","Reading, Video Games, Blogging, Learning","San Francisco, CA",...,24935.0,28.000000,6235.712933,12.000000,6689,6,12486,9,11743,3
1,Rachel B. Moore,10,478090,http://nanowrimo.org/participants/rachel-b-moore,NaN,NaN,"2666, Unaccustomed Earth, Exit Music, Crazy Lo...","Belle and Sebastian, Elliott Smith, PJ Harvey,...","Reading, volunteering, knitting, listening to ...",San Francisco,...,3809.0,9.000000,1002.295167,6.800000,16722,7,24086,14,26517,14
2,abookishbabe,1,0,http://nanowrimo.org/participants/abookishbabe,NaN,April 2,"Colleen Hoover, Veronica Roth, Jennifer Niven,...",Tori Kelley,"Reading (DUH), Day dreaming, Going to Disneyla...","Sacramento, CA",...,NaN,NaN,NaN,NaN,28632,1,29299,2,0,0
3,alexabexis,11,475500,http://nanowrimo.org/participants/alexabexis,NaN,NaN,NaN,Three Goddesses playlist Florence + the Machin...,"drawing, reading, movies & TV shows, comics, p...",New York City,...,2325.0,8.545455,570.626795,8.090909,25360,7,38034,12,40766,9
4,AllYellowFlowers,3,30428,http://nanowrimo.org/participants/AllYellowFlo...,NaN,NaN,"Lolita, Jesus' Son, Ask the",the sound of the coffeemaker,cryptozoology,Allston,...,2054.5,4.500000,538.273315,21.000000,1800,5,5300,10,5700,9


In [2]:
writers.columns

Index([u'Writer Name', u'Member Length', u'LifetimeWordCount', u'url', u'Age',
       u'Birthday', u'Favorite books or authors', u'Favorite noveling music',
       u'Hobbies', u'Location', u'Occupation', u'Primary Role',
       u'Sponsorship URL', u'Expected Final Word Count',
       u'Expected Daily Average', u'CURRENT WINNER', u'Current Donor', u'Wins',
       u'Donations', u'Participated', u'Consecutive Donor',
       u'Consecutive Wins', u'Consecutive Part', u'Part Years', u'Win Years',
       u'Donor Years', u'Num Novels', u'Expected Num Submissions',
       u'Expected Avg Submission', u'Expected Min Submission',
       u'Expected Min Day', u'Expected Max Submission', u'Expected Max Day',
       u'Expected Std Submissions', u'Expected Consec Subs', u'FW Total',
       u'FW Sub', u'FH Total', u'FH Sub', u'SH Total', u'SH Sub'],
      dtype='object')

In [3]:
# convert primary role and sponsorship url to binary vars
writers['Primary Role'][writers['Primary Role'] == 'Municipal Liaison'] = 1
writers['Primary Role'][writers['Primary Role'] != 1] = 0

writers['Sponsorship URL'].fillna(0, inplace=True)
writers['Sponsorship URL'][writers['Sponsorship URL'] != 0] = 1

In [4]:
# let's keep ALL NUMERIC COLUMNS except the CURRENT WINNER column which we will use as response
features = writers._get_numeric_data()
features.columns

Index([u'Member Length', u'LifetimeWordCount', u'Age',
       u'Expected Final Word Count', u'Expected Daily Average',
       u'CURRENT WINNER', u'Current Donor', u'Wins', u'Donations',
       u'Participated', u'Consecutive Donor', u'Consecutive Wins',
       u'Consecutive Part', u'Num Novels', u'Expected Num Submissions',
       u'Expected Avg Submission', u'Expected Min Submission',
       u'Expected Min Day', u'Expected Max Submission', u'Expected Max Day',
       u'Expected Std Submissions', u'Expected Consec Subs', u'FW Total',
       u'FW Sub', u'FH Total', u'FH Sub', u'SH Total', u'SH Sub'],
      dtype='object')

In [5]:
del features['CURRENT WINNER']
# delete features that would only be collected after a contest starts
del features['Current Donor']
del features['FW Total']
del features['FW Sub']
del features['FH Total']
del features['FH Sub']
del features['SH Total']
del features['SH Sub']


features.head()

,Member Length,LifetimeWordCount,Age,Expected Final Word Count,Expected Daily Average,Wins,Donations,Participated,Consecutive Donor,Consecutive Wins,Consecutive Part,Num Novels,Expected Num Submissions,Expected Avg Submission,Expected Min Submission,Expected Min Day,Expected Max Submission,Expected Max Day,Expected Std Submissions,Expected Consec Subs
0,2,50919,24,50919.000000,1697.300000,1,1,1,1,1,1,1,14.000000,3637.071429,299.0,2.000000,24935.0,28.000000,6235.712933,12.000000
1,10,478090,NaN,47809.000000,1593.633333,8,8,10,8,7,10,10,8.300000,918.057453,42.7,7.700000,3809.0,9.000000,1002.295167,6.800000
2,1,0,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,475500,NaN,43227.272727,1440.909091,7,7,11,4,4,11,11,9.272727,822.780595,36.0,6.727273,2325.0,8.545455,570.626795,8.090909
4,3,30428,NaN,15214.000000,507.133333,0,0,2,0,0,1,2,22.000000,678.318083,50.0,10.500000,2054.5,4.500000,538.273315,21.000000


In [6]:
y = writers['CURRENT WINNER'].values

In [7]:
# inputting 0 for users without prior data for daily avg, avg submission, num submissions etc. and so are marked NaN
features.fillna(0, inplace=True)
features.describe()

,Member Length,LifetimeWordCount,Age,Expected Final Word Count,Expected Daily Average,Wins,Donations,Participated,Consecutive Donor,Consecutive Wins,Consecutive Part,Num Novels,Expected Num Submissions,Expected Avg Submission,Expected Min Submission,Expected Min Day,Expected Max Submission,Expected Max Day,Expected Std Submissions,Expected Consec Subs
count,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000,501.000000
mean,4.212575,172552.676647,8.596806,36428.312194,1214.277073,2.606786,1.421158,3.656687,1.047904,1.960080,3.057884,3.377246,10.826177,1708.026777,73.105821,6.128300,4764.389341,10.005534,1314.411102,9.573348
std,3.255209,329113.331830,14.463648,43782.218313,1459.407277,4.651782,3.044384,4.899582,1.760029,2.539764,2.946632,3.451290,8.520344,2053.622361,1566.761571,6.145692,5727.358954,8.406292,2011.241171,8.393503
min,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-21113.500000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,9818.000000,0.000000,7443.250000,248.108333,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,362.750000,0.000000,1.000000,955.000000,1.000000,256.685927,0.000000
50%,4.000000,93385.000000,0.000000,37594.333333,1253.144444,1.000000,0.000000,2.000000,0.000000,1.000000,2.000000,2.000000,10.250000,1446.652778,85.666667,4.500000,3546.500000,9.333333,873.018486,8.500000
75%,6.000000,206482.000000,20.000000,50734.200000,1691.140000,3.000000,2.000000,5.000000,1.000000,3.000000,4.000000,5.000000,17.333333,2213.520000,291.500000,10.000000,6250.000000,16.200000,1516.145753,16.000000
max,13.000000,4562712.000000,61.000000,651816.000000,21727.200000,52.000000,36.000000,52.000000,9.000000,14.000000,14.000000,26.000000,30.000000,20869.236584,5000.000000,27.666667,51238.000000,30.000000,23874.872328,30.000000


In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from bokeh.plotting import figure,show,output_notebook
from bokeh.models import Range1d
output_notebook()

BokehJS successfully loaded.

### Normalize data

In [45]:
scaler = StandardScaler()
features_norm = scaler.fit_transform(features)
features_norm[1]

array([ 1.77967343,  0.92929298, -0.5949674 ,  0.26019839,  0.26019839,
        1.16054543,  2.1631367 ,  1.29595831,  3.95393831,  1.98638802,
        2.35830378,  1.92083955, -0.29678396, -0.38505565, -0.01942619,
        0.25599572, -0.16697823, -0.11973645, -0.15534084, -0.33074635])

### Apply Logistic Regression

In [57]:
def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = auc(fpr, tpr)
    
    p = figure(title='Receiver Operating Characteristic')
    # Plot ROC curve
    p.line(x=fpr,y=tpr,legend='ROC curve (area = %0.3f)' % roc_auc)
    p.x_range=Range1d(0,1)
    p.y_range=Range1d(0,1)
    p.xaxis.axis_label='False Positive Rate or (1 - Specifity)'
    p.yaxis.axis_label='True Positive Rate or (Sensitivity)'
    p.legend.location = "bottom_right"
    show(p)
    
%matplotlib inline

In [58]:
X_train, X_test, y_train, y_test = train_test_split(features_norm,y, test_size=0.2, random_state=0)

In [59]:
model_lr = LogisticRegression(C=5)
print cross_val_score(model_lr,X_train, y_train,cv=10).mean()

model_lr.fit(X_train, y_train)
print pd.DataFrame(confusion_matrix(y_test,model_lr.predict(X_test)), index=['Predicted Class 0', 'Predicted Class 1'], 
                     columns=['Actual Class 0', 'Actual Class 1'])
print classification_report(y_test,model_lr.predict(X_test))
print model_lr.score(X_test,y_test)
plot_roc_curve(y_test, model_lr.predict_proba(X_test))


0.682823639775
                   Actual Class 0  Actual Class 1
Predicted Class 0              48               7
Predicted Class 1              23              23
             precision    recall  f1-score   support

          0       0.68      0.87      0.76        55
          1       0.77      0.50      0.61        46

avg / total       0.72      0.70      0.69       101

0.70297029703


This is not as accurate as when including current contest data.  We can assume then that activity in the first couple weeks of the contest is predictive of winning.  

Still, let's try some other models and see how they do.



### Naive Bayes

In [74]:
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()
print cross_val_score(model_nb, X_train, y_train, cv=10).mean()

model_nb.fit(X_train, y_train)
print pd.DataFrame(confusion_matrix(y_test,model_nb.predict(X_test)), index=['Predicted Class 0', 'Predicted Class 1'], 
                     columns=['Actual Class 0', 'Actual Class 1'])
print classification_report(y_test,model_nb.predict(X_test))
print model_nb.score(X_test,y_test)

0.680079737336
                   Actual Class 0  Actual Class 1
Predicted Class 0              45              10
Predicted Class 1              19              27
             precision    recall  f1-score   support

          0       0.70      0.82      0.76        55
          1       0.73      0.59      0.65        46

avg / total       0.72      0.71      0.71       101

0.712871287129


Naive Bayes is not as accurate as Logistic Regression in this case.

### SVM

In [84]:
from sklearn.svm import SVC

model_svc = SVC(kernel="rbf",C=5)
print cross_val_score(model_svc, X_train, y_train, cv=10).mean()

model_svc.fit(X_train, y_train)
print pd.DataFrame(confusion_matrix(y_test,model_svc.predict(X_test)), index=['Predicted Class 0', 'Predicted Class 1'], 
                     columns=['Actual Class 0', 'Actual Class 1'])
print classification_report(y_test,model_svc.predict(X_test))
print model_svc.score(X_test,y_test)

0.690073483427
                   Actual Class 0  Actual Class 1
Predicted Class 0              45              10
Predicted Class 1              20              26
             precision    recall  f1-score   support

          0       0.69      0.82      0.75        55
          1       0.72      0.57      0.63        46

avg / total       0.71      0.70      0.70       101

0.70297029703


This Support Vector Machine does a little bit better than the Logistic Regression.

### Decision Tree

In [85]:
from sklearn.tree import DecisionTreeClassifier

In [86]:
model_dt = DecisionTreeClassifier(max_depth=5)
print cross_val_score(model_dt, X_train, y_train, cv=10).mean()

model_dt.fit(X_train, y_train)
print pd.DataFrame(confusion_matrix(y_test,model_dt.predict(X_test)), index=['Predicted Class 0', 'Predicted Class 1'], 
                     columns=['Actual Class 0', 'Actual Class 1'])
print classification_report(y_test,model_dt.predict(X_test))
print model_dt.score(X_test,y_test)

0.627926829268
                   Actual Class 0  Actual Class 1
Predicted Class 0              38              17
Predicted Class 1              19              27
             precision    recall  f1-score   support

          0       0.67      0.69      0.68        55
          1       0.61      0.59      0.60        46

avg / total       0.64      0.64      0.64       101

0.643564356436


The Decision Tree did not do as well this time without the other features.

In [87]:
dt_importances = pd.DataFrame(zip(features.columns, model_dt.feature_importances_))
dt_importances.sort_values(1, ascending=False).head() # most to least predictive of being 0

,0,1
4,Expected Daily Average,0.405392
9,Consecutive Wins,0.099488
1,LifetimeWordCount,0.077530
19,Expected Consec Subs,0.075867
14,Expected Min Submission,0.073147


Without the data from the current contest, the most important features are Expected Daily Average and LifetimeWordCount, or a writer's average daily writing productivity and how much they've participated in the past.

### Random Forests

In [88]:
from sklearn.ensemble import RandomForestClassifier

In [89]:
model_rf = RandomForestClassifier(max_depth=5, n_estimators=100, max_features=2)
print cross_val_score(model_rf, X_train, y_train, cv=10).mean()

model_rf.fit(X_train, y_train)
print pd.DataFrame(confusion_matrix(y_test,model_rf.predict(X_test)), index=['Predicted Class 0', 'Predicted Class 1'], 
                     columns=['Actual Class 0', 'Actual Class 1'])
print classification_report(y_test,model_rf.predict(X_test))
print model_rf.score(X_test,y_test)

0.690326766729
                   Actual Class 0  Actual Class 1
Predicted Class 0              46               9
Predicted Class 1              18              28
             precision    recall  f1-score   support

          0       0.72      0.84      0.77        55
          1       0.76      0.61      0.67        46

avg / total       0.74      0.73      0.73       101

0.732673267327


It looks like Random Forests and Support Vector Machines do best in predicting winners and non-winners when excluding data from the current contest.